In [7]:
import matplotlib.pyplot as plt;
import pandas as pd
import numpy as np
from sklearn import preprocessing
from kneed import DataGenerator, KneeLocator
from sklearn.metrics import accuracy_score
from sklearn import metrics
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.datasets import make_classification
from sklearn.naive_bayes import GaussianNB
from sklearn.calibration import CalibratedClassifierCV
from sklearn.naive_bayes import BernoulliNB

In [2]:
path=r''
traData = pd.read_csv(path+'trainingGroup.csv',index_col=None, header=None)
#traData = pd.read_csv('D:\\drive\\publications_2017\\datasets\\NSL-KDD\\NSL-KDD\\testingData.csv'  , index_col = None , header = None)
tesData = pd.read_csv(path+'testingGroup.csv',index_col=None, header=None)
target = 41

y_train = traData[[target]].values.reshape(len(traData)) 
x_train = preprocessing.normalize(traData.iloc[:,:-1])

y_test = tesData[[target]].values.reshape(len(tesData)) 
x_test = preprocessing.normalize(tesData.iloc[:,:-1])

def chooseObsMan(dic_f):
    chosenIDs=np.array(())
    for k,d in dic_f.items():
        temp=np.where(y_train==k)[0]
        z=np.random.choice(temp,dic_f[k],replace=False)
        chosenIDs=np.concatenate((chosenIDs,z))
    return np.int64(chosenIDs)
def label_index(row,threshold=0):
    
    if threshold==0:
        threshold=np.max(row)
    temp_i=np.where(row>=threshold)[0]
    if temp_i.shape[0]>0:
        return temp_i[0]+1
    else:
        return -1

labelled=np.zeros(x_train.shape[0])
labelled[:]=-1
li={1:60,2:60,3:30,4:10,5:10}
labelledIndex=chooseObsMan(li)
labelled[labelledIndex]=y_train[labelledIndex]

In [8]:
clf = BernoulliNB()
clf.fit(x_train, y_train)

BernoulliNB()

In [15]:
predicted = clf.predict(x_train)

In [17]:
print((predicted == y_train).mean())

0.8153175680502965


In [18]:
matrices = metrics.precision_recall_fscore_support(y_train, predicted,labels=[1, 2, 3, 4, 5])
resultsDataFrame = pd.DataFrame()
for i in range(0,len(matrices)):
    temp = pd.DataFrame([matrices[i]],columns=['normal','DoS','Probe','R2L','U2R'])
    resultsDataFrame = resultsDataFrame.append(temp, ignore_index=True)
resultsDataFrame.insert(0,'  ',['Precision','Recall','Fscore','Support'])
print(resultsDataFrame)

                    normal           DoS         Probe         R2L        U2R
0  Precision      0.904502      0.962465      0.294147    0.306367   0.144144
1     Recall      0.907162      0.766564      0.512183    0.411055   0.615385
2     Fscore      0.905830      0.853417      0.373686    0.351073   0.233577
3    Support  67343.000000  45927.000000  11656.000000  995.000000  52.000000


In [19]:
correctLabelled=metrics.accuracy_score(y_train, predicted,normalize=False)
incorrectLabelled=sum(resultsDataFrame.iloc[3,1:]) - correctLabelled
print([correctLabelled,incorrectLabelled])

[102708, 23265.0]


In [20]:
numbers=resultsDataFrame.iloc[3,1:]
correctLabelledForEachclass=resultsDataFrame.iloc[3,1:] *resultsDataFrame.iloc[1,1:]
incorrectLabelledForEachclass=resultsDataFrame.iloc[3,1:]-correctLabelledForEachclass
results1 = pd.DataFrame(columns = ['Total','Correct', 'Incorrect'])
results1['Total'] = numbers
results1['Correct'] = correctLabelledForEachclass
results1['Incorrect'] = incorrectLabelledForEachclass
print(results1)


        Total Correct Incorrect
normal  67343   61091      6252
DoS     45927   35206     10721
Probe   11656    5970      5686
R2L       995     409       586
U2R        52      32        20
